# Merge all data sets

Earnings from yahoo.com, metrics from stockrow.com, and sector/industries from marketwatch.com

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tickerlist import make_list
import re
import pickle
import json

## Clean sector and industry data set

In [2]:
sectors = {'consumer_discr': ['Computers/Consumer Electronics', 'Mixed Retailing', 'Home Goods Retail', 'Broadcasting', 'Restaurants', 'Tourism', 'Footwear', 'Automobiles', 'Toys & Games', 'Clothing Retail', 'Hotels', 'Auto & Commercial Vehicle Parts', 'Commercial Vehicles', 'Housewares', 'Specialty Retail', 'Clothing', 'Advertising/Marketing/Public Relations', 'Publishing', 'Gambling Industries', 'Residential Building Construction', 'Tires', 'Furniture'],
           'conumser_staples': ['Personal Care Products/Appliances', 'Non-Alcoholic Beverages/Drinks', 'Tobacco', 'Drug Retail', 'Food Retail', 'Food Products', 'Nondurable Household Products', 'Alcoholic Beverages/Drinks'],
           'energy': ['Major Oil & Gas', 'Oil & Gas Products/Services', 'Oil Extraction', 'Pipeline Transportation'],
           'financials': ['Reinsurance', 'Major International Banks', 'Banking', 'Diversified Holding Companies', 'Consumer Finance', 'Life Insurance', 'Securities', 'Finance Companies', 'Non-Life Insurance', 'Investment Advisors', 'Full-Line Insurance', 'Accounting', 'Insurance Brokering', 'Savings Institutions'],
           'health_care': ['Pharmaceuticals', 'Biotechnology', 'Medical Equipment/Supplies', 'Precision Products', 'Healthcare Provision', ],
           'industrials': ['Aerospace Products/Parts', 'Railroads', 'Air Freight', 'Defense Equipment/Products', 'Mobile Machinery', 'Farming', 'Industrial Machinery', 'Passenger Airlines', 'Environment/Waste Management', 'Industrial Products', 'Building Materials/Products', 'Wholesalers', 'General Services', 'Transportation Services', 'Trucking', 'Technical Services', 'Employment/Training Services', 'Construction'],
           'info_tech': ['Software', 'Internet/Online', 'Semiconductors', 'Networking', 'Computer Services', 'Diversified Business Services', 'Industrial Electronics'],
           'materials': ['Chemicals', 'Paper/Pulp', 'Gold', 'Non-Ferrous Metals', 'Iron/Steel', 'Containers/Packaging'],
           'real_estate': ['Specialty REITs', 'Retail REITs', 'Industrial/Office REITs', 'Healthcare REITs', 'Residential REITs', 'Hotel/Lodging REITs', 'Diversified REITs', 'Real Estate Developers'],
           'telecomm': ['Wireless Telecommunications Services', 'Wired Telecommunications Services'],
           'utilities': ['Electric Utilities', 'Multiutilities', 'Gas Utilities', 'Water Utilities']}

### Replace marketwatch sectors with official GISC sectors

In [3]:
with open('/Users/samfunk/ds/metis/project_mcnulty/datafiles/sector_industries.pkl', 'rb') as f:
    sect_ind_dict = pickle.load(f)

sector_industry_raw = pd.DataFrame.from_dict(sect_ind_dict, orient='index')
sect_ind_df = pd.DataFrame(columns=['mw_sector', 'industry', 'true_sector'])

for ticker in sector_industry_raw.index:
    true_sector = [k for k,v in sectors.items() if sector_industry_raw.loc[ticker][1] in v]
    for i in range(1,5):
        sect_ind_df.loc['%s_%d' % (ticker, i)] = sector_industry_raw.loc[ticker].tolist() + true_sector

In [4]:
len(sect_ind_df)

1984

In [5]:
sect_ind_df.true_sector.value_counts()

consumer_discr      392
financials          340
health_care         228
industrials         212
info_tech           208
conumser_staples    148
real_estate         128
utilities           116
energy              116
materials            80
telecomm             16
Name: true_sector, dtype: int64

## Merge time

In [6]:
with open('/Users/samfunk/ds/metis/project_mcnulty/datafiles/earnings_df.pkl', 'rb') as f :
    earnings_df = pickle.load(f)

with open('/Users/samfunk/ds/metis/project_mcnulty/datafiles/metrics.pkl', 'rb') as f:
    metrics_df = pickle.load(f)

master = metrics_df.join(earnings_df, how='left')
master = master.join(sect_ind_df, how='left')

### Fill and remove various nans

In [7]:
master = master.reset_index()
master.rename(columns={'index': 'ticker_quarter'}, inplace=True)
master.loc[(master.currentR.isnull()) & (master.interest == 0), 'interest'] = 7.0
master['currentR'] = master['currentR'].fillna(1)
master['payout'] = master['payout'].fillna(0)
master.dropna(axis=0, inplace=True)

In [8]:
master

,ticker_quarter,date,bookvalue,fcf,intdebt,cash,debt_equity,interest,currentR,incomeQ,...,rd,difference,surprise,above_below,abs_surprise,abslog_surprise,buckets,mw_sector,industry,true_sector
0,AAPL_1,2017-07-01,25.490,1.171,20.8541,3.5747,0.8181,0.0000,1.388,0.9594,...,0.0647,-0.16,-9.6,-1.0,9.6,2.261763,1.0,Technology,Computers/Consumer Electronics,consumer_discr
1,AAPL_2,2017-04-01,25.658,1.827,18.8530,2.9004,0.7348,0.0000,1.391,1.1355,...,0.0525,0.15,4.7,1.0,4.7,1.547563,0.0,Technology,Computers/Consumer Electronics,consumer_discr
2,AAPL_3,2016-12-31,24.986,4.477,16.5229,3.0896,0.6613,0.0000,1.228,1.5123,...,0.0366,0.08,4.0,1.0,4.0,1.386294,0.0,Technology,Computers/Consumer Electronics,consumer_discr
3,AAPL_4,2016-09-24,23.896,2.264,16.2164,3.8167,0.6786,0.0000,1.353,1.7890,...,0.0549,0.10,6.4,1.0,6.4,1.856298,1.0,Technology,Computers/Consumer Electronics,consumer_discr
4,MSFT_1,2017-06-30,9.382,1.130,11.1708,17.2344,1.1906,0.0000,2.477,1.6897,...,0.1507,0.04,5.9,1.0,5.9,1.774952,1.0,Technology,Software,info_tech
5,MSFT_2,2017-03-31,9.026,1.161,10.8748,16.3130,1.2048,0.0000,2.813,2.2204,...,0.1519,0.01,1.3,1.0,1.3,0.262364,0.0,Technology,Software,info_tech
6,MSFT_3,2016-12-31,8.873,0.555,10.9568,15.8325,1.2349,0.0000,2.048,1.2102,...,0.1271,0.00,0.0,0.0,0.0,-inf,0.0,Technology,Software,info_tech
7,MSFT_4,2016-09-30,9.035,1.205,9.5892,17.5802,1.0614,0.0000,2.685,2.4625,...,0.1519,0.35,49.3,1.0,49.3,3.897924,1.0,Technology,Software,info_tech
8,FB_1,2017-06-30,22.924,1.350,0.0000,2.1559,0.0000,0.0000,12.309,1.3779,...,0.2059,0.12,12.4,1.0,12.4,2.517696,1.0,Technology,Internet/Online,info_tech
9,FB_2,2017-03-31,21.511,1.310,0.0000,2.4573,0.0000,0.0000,12.613,1.6535,...,0.2283,0.10,7.6,1.0,7.6,2.028148,1.0,Technology,Internet/Online,info_tech


## Segment various features

### Free Cash Flow

In [9]:
def fcf(row):
    if row['fcf'] < 0:
        return 0
    elif row['fcf'] < 1:
        return 1
    elif row['fcf'] < 2:
        return 2
    else:
        return 3

master['free_cash_flow'] = master.apply(lambda x: fcf(x), axis=1)

In [10]:
master.free_cash_flow.value_counts()

1    759
2    442
0    372
3    363
Name: free_cash_flow, dtype: int64

### Dividends (payout ratio)

In [11]:
def divs(row):
    if row['payout'] < 0:
        return 0
    elif row['payout'] == 0:
        return 1
    else:
        return 2

master['dividends'] = master.apply(lambda x: divs(x), axis=1)

In [12]:
master.dividends.value_counts()

2    1419
1     397
0     120
Name: dividends, dtype: int64

### Interest covereage (industry standard = 1.5)

In [13]:
def interest_coverage(row):
    if row['interest'] <= 1.5:
        return 0
    else:
        return 1
master.loc[master['interest'] == 0, ]
master['interest_coverage'] = master.apply(lambda x: interest_coverage(x), axis=1)

In [14]:
master.interest_coverage.value_counts()

1    1602
0     334
Name: interest_coverage, dtype: int64

### Current ratio (industry standard < 1)

In [15]:
missing_currents = {'BRKB': 1.3,'GE': 2.16,'AXP': 1.96,'MET': 0.98,'SCHW': 0.33,'SPG': 0.27,'PRU': 1.19,'CI': 0.90,'AIG': 2.82,'DE': 2.12,'TRV': 0.40,'PLD': 1.14,'ALL': 0.45,'AFL': 0.40,'PSA': 1.11,'PGR': 1.00,'DLR': 0.77,'HCN': 1.53,'EQR': 0.26,'AVB': 0.92,'DFS': 1.53,'AMP': 2.58,'TROW': 3.36,'VTR': 0.92,'HIG': 4.35,'BXP': 2.96,'PFG': 2.66,'LNC': 3.24,'ESS': 0.70,'O': 0.61,'DHI': 7.22,'VNO': 6.28,'L': 0.52,'GGP': 1.58,'ETFC': 1.40,'HCP': 4.01,'MAA': 0.17,'UNM': 8.98,'CINF': 1.22,'LEN': 6.62,'X': 0.50,'ARE': 1.45,'RJF': 3.31,'SLG': 4.16,'DRE': 10.30,'UDR': 0.18,'EXR': 0.99,'RE': 2.00,'REG': 1.37,'FRT': 1.40,'WU': 0.29,'TMK': 0.32,'KIM': 2.57,'PHM': 4.27,'AIV': 0.18,'MAC': 0.68,'AIZ': 0.93,'NAVI': 34.22,
}

for x in master['ticker_quarter']:
    ticker = re.search(r'([A-Z]*)', x)[1]
    if ticker in missing_currents.keys():
        master.loc[master['ticker_quarter'] == x, 'currentR'] = missing_currents[ticker]
        
def current_ratio(row):
    if row['currentR'] < 1:
        return 0
    else:
        return 1

master['current_ratio'] = master.apply(lambda x: current_ratio(x), axis=1)

In [16]:
master.current_ratio.value_counts()

1    1480
0     456
Name: current_ratio, dtype: int64

### Debt to Equity ratio (industry standard = 2)

In [17]:
def debt_to_equity(row):
    if row['debt_equity'] < 0:
        return 0
    elif row['debt_equity'] < 0.5:
        return 1
    elif row['debt_equity'] < 1:
        return 2
    else:
        return 3

master['debt_to_equity'] = master.apply(lambda x: debt_to_equity(x), axis=1)

In [18]:
master.debt_to_equity.value_counts()

3    768
2    590
1    506
0     72
Name: debt_to_equity, dtype: int64

### Quality of income/earnings (industry standard ~ 1)

In [19]:
def income_quality(row):
    if row['incomeQ'] < 1:
        return 0
    else:
        return 1

master['income_quality'] = master.apply(lambda x: income_quality(x), axis=1)

In [20]:
master.income_quality.value_counts()

1    1354
0     582
Name: income_quality, dtype: int64

### Research and Development as percentage of revenue (greater than or equal to 0)

In [21]:
def r_and_d(row):
    if row['rd'] <= 0:
        return 0
    else:
        return 1

master['r_and_d'] = master.apply(lambda x: r_and_d(x), axis=1)

In [22]:
master.r_and_d.value_counts()

0    1381
1     555
Name: r_and_d, dtype: int64

### Save final/master df

In [ ]:
with open('/Users/samfunk/ds/metis/project_mcnulty/datafiles/master_df.pkl', 'wb') as f:
    pickle.dump(master, f)

In [ ]:
master